In [1]:
%pip install -U langchain-community langgraph langchain-openai tavily-python

Note: you may need to restart the kernel to use updated packages.


In [14]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [15]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [16]:
from langchain_community.tools.tavily_search import TavilySearchResults

# define search tool

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1721082613, 'localtime': '2024-07-15 15:30'}, 'current': {'last_updated_epoch': 1721081700, 'last_updated': '2024-07-15 15:15', 'temp_c': 21.7, 'temp_f': 71.1, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 15.0, 'wind_kph': 24.1, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1017.0, 'pressure_in': 30.04, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 55, 'cloud': 25, 'feelslike_c': 21.7, 'feelslike_f': 71.1, 'windchill_c': 17.5, 'windchill_f': 63.5, 'heatindex_c': 17.5, 'heatindex_f': 63.5, 'dewpoint_c': 11.9, 'dewpoint_f': 53.5, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 19.5, 'gust_kph': 31.3}}"}, {'url': 'https://www.wunderground.com/hourly

In [5]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [17]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")

In [18]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [19]:
# bind tools with model
model_with_tools = model.bind_tools(tools)

In [20]:
# check response of model with search tool (no tool call)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [21]:
# check response of model with search tool (tool call)
response = model_with_tools.invoke([HumanMessage(content="What's the weather in new york city?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in New York City'}, 'id': 'call_BmQGIBDukSRhioaK2pzNkgae', 'type': 'tool_call'}]


In [11]:
from langgraph.prebuilt import create_react_agent

#use prebuilt ReAct agent
agent_executor = create_react_agent(model, tools)

In [12]:
# test agent response (no tool call)
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='3749604e-ea44-4099-9a58-755ac1ed5216'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_dd932ca5d1', 'finish_reason': 'stop', 'logprobs': None}, id='run-5522f594-1aaf-49ef-bd2f-e2d1f94d1b2a-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [13]:
# test agent response (tool call)
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in new york city?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in new york city?', id='37f6d006-2e4b-4ce6-b6a6-e894f425806f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MNa2XEcpF11HAMxhXL0wbl4b', 'function': {'arguments': '{"query":"current weather in New York City"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_d33f7b429e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-eee4b69c-a60a-4a42-aa4b-50b6888be4f5-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in New York City'}, 'id': 'call_MNa2XEcpF11HAMxhXL0wbl4b', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'New York\', \'region\': \'New York\'